In [32]:
# Installing Modules/Packages
!pip install audiosegment
!pip install --upgrade google-cloud-speech
!pip install --upgrade google-cloud-texttospeech

In [33]:
# Importing Modules/Packages
from google.colab import drive
from pydub import AudioSegment
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1 import types
import os
import io
from scipy.io import wavfile
import os
import json
from google.cloud import translate
from google.cloud import texttospeech
import html
import shutil
import tempfile
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip, TextClip

In [34]:
# connecting to google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Setting Credentials to access APIs
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/My Drive/GCP/Speech-to-text.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/My Drive/GCP/translationAPI.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/My Drive/GCP/text-to-speech-practice.json'

In [36]:
# Extracting audio from Video
def decode_audio(inFile, outFile):
  '''
    infile: input file path i.e. a\b\c.mp4
    outfile: output path i.e. a\b\c.wav
  '''
  if not outFile[-4:] != "wav":
    outFile += ".wav"
  AudioSegment.from_file(inFile).set_channels(1).export(outFile, format="wav")

In [37]:
def get_transcripts_json(gdrivePath, langCode, phraseHints=[], speakerCount=1, enhancedModel=None):
    """Transcribes audio files.
    Args:
        gdrivePath (String): path to file in cloud storage (i.e. "/content/drive/My Drive/audio/clip.mp4")
        langCode (String): language code (i.e. "en-US", see https://cloud.google.com/speech-to-text/docs/languages)
        phraseHints (String[]): list of words that are unusual but likely to appear in the audio file.
        speakerCount (int, optional): Number of speakers in the audio. Only works on English. Defaults to None.
        enhancedModel (String, optional): Option to use an enhanced speech model, i.e. "video"
    Returns:
        list | Operation.error
    """

    # Helper function for simplifying Google speech client response
    def _jsonify(result):
        json = []
        for section in result.results:
            data = {
                "transcript": section.alternatives[0].transcript,
                "words": []
            }
            for word in section.alternatives[0].words:
                data["words"].append({
                    "word": word.word,
                    "start_time": word.start_time.total_seconds(),
                    "end_time": word.end_time.total_seconds(),
                    "speaker_tag": word.speaker_tag
                })
            json.append(data)
        return json


    def get_audio(gdrivePath):
        with io.open(gdrivePath, "rb") as audio_file:
            content = audio_file.read()
            audio = speech.RecognitionAudio(content=content)
        return audio
    client = speech.SpeechClient()  

    # audio = speech.RecognitionAudio(uri=gdrivePath)
    # audio = speech.RecognitionAudio(uri=get_audio(gdrivePath))
    audio = get_audio(gdrivePath)

    diarize = speakerCount if speakerCount > 1 else False
    print(f"Diarizing: {diarize}")
    diarizationConfig = speech.SpeakerDiarizationConfig(
        enable_speaker_diarization=speakerCount if speakerCount > 1 else False,
    )

    # In English only, we can use the optimized video model
    if langCode == "en":
        enhancedModel = "video"

    config = speech.RecognitionConfig(
        language_code="en-US" if langCode == "en" else langCode,
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,
        speech_contexts=[{
            "phrases": phraseHints,
            "boost": 15
        }],
        diarization_config=diarizationConfig,
        profanity_filter=True,
        use_enhanced=True if enhancedModel else False,
        model="video" if enhancedModel else None

    )
    res = client.long_running_recognize(config=config, audio=audio).result()

    return _jsonify(res)

In [38]:
def parse_sentence_with_speaker(json, lang):
    """Takes json from get_transcripts_json and breaks it into sentences
    spoken by a single person. Sentences deliniated by a >= 1 second pause/
    Args:
        json (string[]): [{"transcript": "lalala", "words": [{"word": "la", "start_time": 20, "end_time": 21, "speaker_tag: 2}]}]
        lang (string): language code, i.e. "en"
    Returns:
        string[]: [{"sentence": "lalala", "speaker": 1, "start_time": 20, "end_time": 21}]
    """

    # Special case for parsing japanese words
    def get_word(word, lang):
        if lang == "ja":
            return word.split('|')[0]
        return word

    sentences = []
    sentence = {}
    for result in json:
        for i, word in enumerate(result['words']):
            wordText = get_word(word['word'], lang)
            if not sentence:
                sentence = {
                    lang: [wordText],
                    'speaker': word['speaker_tag'],
                    'start_time': word['start_time'],
                    'end_time': word['end_time']
                }
            # If we have a new speaker, save the sentence and create a new one:
            elif word['speaker_tag'] != sentence['speaker']:
                sentence[lang] = ' '.join(sentence[lang])
                sentences.append(sentence)
                sentence = {
                    lang: [wordText],
                    'speaker': word['speaker_tag'],
                    'start_time': word['start_time'],
                    'end_time': word['end_time']
                }
            else:
                sentence[lang].append(wordText)
                sentence['end_time'] = word['end_time']

            # If there's greater than one second gap, assume this is a new sentence
            if i+1 < len(result['words']) and word['end_time'] < result['words'][i+1]['start_time']:
                sentence[lang] = ' '.join(sentence[lang])
                sentences.append(sentence)
                sentence = {}
        if sentence:
            sentence[lang] = ' '.join(sentence[lang])
            sentences.append(sentence)
            sentence = {}

    return sentences


In [39]:
# Transcribing transcript to another language (input: transcript, language(output))
def translate_text(input, targetLang, sourceLang=None):
    """Translates from sourceLang to targetLang. If sourceLang is empty,
    it will be auto-detected.
    Args:
        sentence (String): Sentence to translate
        targetLang (String): i.e. "en"
        sourceLang (String, optional): i.e. "es" Defaults to None.
    Returns:
        String: translated text
    """

    translate_client = translate.Client()
    result = translate_client.translate(input, target_language=targetLang, source_language=sourceLang)

    return html.unescape(result['translatedText'])

In [40]:
def speak(text, languageCode, voiceName=None, speakingRate=1):
  """Converts text to audio
    Args:
        text (String): Text to be spoken
        languageCode (String): Language (i.e. "en")
        voiceName: (String, optional): See https://cloud.google.com/text-to-speech/docs/voices
        speakingRate: (int, optional): speed up or slow down speaking range [0.25, 4.0]
    Returns:
        bytes : Audio in wav format
    """
  # Instantiates a client
  client = texttospeech.TextToSpeechClient()

  # Set the text input to be synthesized
  synthesis_input = texttospeech.SynthesisInput(text=text)
  
  # Build the voice request, select the language code ("en-US") and the ssml
  # voice gender ("neutral")
  if not voiceName:
      voice = texttospeech.VoiceSelectionParams(
          language_code=languageCode, ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL # If input voice name is not provided
      )
  else:
      voice = texttospeech.VoiceSelectionParams(
          language_code=languageCode, name=voiceName # If input voice name is provided
      )

  # Select the type of audio file you want returned
  audio_config = texttospeech.AudioConfig(
      audio_encoding=texttospeech.AudioEncoding.MP3,
      speaking_rate=speakingRate
  )
  
  # Perform the text-to-speech request on the text input with the selected
  # voice parameters and audio file type
  response = client.synthesize_speech(
      input=synthesis_input,
      voice=voice,
      audio_config=audio_config
  )

  return response.audio_content

In [41]:
def speakUnderDuration(text, languageCode, durationSecs, voiceName=None):
    """Speak text within a certain time limit.
    If audio already fits within duratinSecs, no changes will be made.
    Args:
        text (String): Text to be spoken
        languageCode (String): language code, i.e. "en"
        durationSecs (int): Time limit in seconds
        voiceName (String, optional): See https://cloud.google.com/text-to-speech/docs/voices
    Returns:
        bytes : Audio in wav format
    """
    baseAudio = speak(text, languageCode, voiceName=voiceName)
    f = tempfile.NamedTemporaryFile(mode="w+b")
    f.write(baseAudio)
    f.flush()
    baseDuration = AudioSegment.from_mp3(f.name).duration_seconds
    f.close()
    ratio = baseDuration / durationSecs

    # if the audio fits, return it
    if ratio <= 1:
        return baseAudio

    # If the base audio is too long to fit in the segment...

    # round to one decimal point and go a little faster to be safe,
    ratio = round(ratio, 1)
    if ratio > 4:
        ratio = 4
    return speak(text, languageCode, voiceName=voiceName, speakingRate=ratio)

In [42]:
def stitch_audio(sentences, audioDir, movieFile, outFile, overlayGain=-30):
    """Combines sentences, audio clips, and video file into the ultimate dubbed video
    Args:
        sentences (list): Output of parse_sentence_with_speaker
        audioDir (String): Directory containing generated audio files to stitch together
        movieFile (String): Path to movie file to dub.
        outFile (String): Where to write dubbed movie.
        overlayGain (int, optional): How quiet to make source audio when overlaying dubs. 
            Defaults to -30.
    Returns:
       void : Writes movie file to outFile path
    """

    # Files in the audioDir should be labeled 0.wav, 1.wav, etc.
    audioFiles = os.listdir(audioDir)
    audioFiles.sort(key=lambda x: int(x.split('.')[0]))

    # Grab the computer-generated audio file
    segments = [AudioSegment.from_mp3(
        os.path.join(audioDir, x)) for x in audioFiles]
    # Also, grab the original audio
    dubbed = AudioSegment.from_file(movieFile)

    # Place each computer-generated audio at the correct timestamp
    for sentence, segment in zip(sentences, segments):
        dubbed = dubbed.overlay(
            segment, position=sentence['start_time'] * 1000, gain_during_overlay=overlayGain)
    # Write the final audio to a temporary output file
    audioFile = tempfile.NamedTemporaryFile()
    dubbed.export(audioFile)
    audioFile.flush()

    # Add the new audio to the video and save it
    clip = VideoFileClip(movieFile)
    audio = AudioFileClip(audioFile.name)
    clip = clip.set_audio(audio)

    clip.write_videofile(outFile, codec='libx264', audio_codec='aac')
    audioFile.close()

In [43]:
def dub(videoFile, outputDir, srcLang, targetLangs=[], phraseHints=[], speakerCount=1, genAudio=True, voices={}):
  """Translate and dub a movie.
#     Args:
#         videoFile (String): File to dub
#         outputDir (String): Directory to write output files
#         srcLang (String): Language code to translate from (i.e. "fi")
#         targetLangs (list, optional): Languages to translate too, i.e. ["en", "fr"]
#         phraseHints (list, optional): "Hints" for words likely to appear in audio. Defaults to [].
#         speakerCount (int, optional): How many speakers in the video. Defaults to 1.
#         voices (dict, optional): Which voices to use for dubbing, i.e. {"en": "en-AU-Standard-A"}. Defaults to {}.
#         genAudio (bool, optional): Generate new audio, even if it's already been generated. Defaults to False.
#     Raises:
#         void : Writes dubbed video and intermediate files to outputDir
#     """
  baseName = os.path.split(videoFile)[-1].split('.')[0]
  outputFiles = os.listdir(outputDir)
  if not f"{baseName}.wav" in outputFiles:
        print("Extracting audio from video")
        fn = os.path.join(outputDir, baseName + ".wav")
        print(fn)
        decode_audio(videoFile, fn)
        print(f"Wrote {fn}")
        if not f"transcript.json" in outputFiles:
          print("Connecting to Drive")
          print("Connected to Drive")
          print("Transcribing audio")
          transcripts = get_transcripts_json(fn, srcLang, phraseHints=phraseHints, speakerCount=speakerCount)
          json.dump(transcripts, open(os.path.join(outputDir, "transcript.json"), "w"))
          print(transcripts)
          sentences = parse_sentence_with_speaker(transcripts, srcLang)
          fn = os.path.join(outputDir, baseName + ".json")
          with open(fn, "w") as f:
              json.dump(sentences, f)
          print(f"Wrote {fn}")
          sentences = json.load(open(os.path.join(outputDir, baseName + ".json")))
          sentecnce = sentences[0]
          for lang in targetLangs:
            print(f"Translating to {lang}")
            for sentence in sentences:
                sentence[lang] = translate_text(
                    sentence[srcLang], lang, srcLang)

          # Write the translations to json
          fn = os.path.join(outputDir, baseName + ".json")
          with open(fn, "w") as f:
              json.dump(sentences, f)
          print(f"Wrote {fn}")
  sentences = json.load(open(os.path.join(outputDir, baseName + ".json")))
  sentence = sentences[0]
  audioDir = os.path.join(outputDir, "audioClips")
  if not "audioClips" in outputFiles:
      os.mkdir(audioDir)
  for lang in targetLangs:
      languageDir = os.path.join(audioDir, lang)
      if os.path.exists(languageDir):
          if not genAudio:
              continue
          shutil.rmtree(languageDir)
      os.mkdir(languageDir)
      print(f"Synthesizing audio for {lang}")
      for i, sentence in enumerate(sentences):
          voiceName = voices[lang] if lang in voices else None
          audio = speakUnderDuration(sentence[lang], lang, sentence['end_time'] - sentence['start_time'], voiceName=voiceName)
          with open(os.path.join(languageDir, f"{i}.mp3"), 'wb') as f:
              f.write(audio)
  dubbedDir = os.path.join(outputDir, "dubbedVideos")

  if not "dubbedVideos" in outputFiles:
      os.mkdir(dubbedDir)
  for lang in targetLangs:
        print(f"Dubbing audio for {lang}")
        outFile = os.path.join(dubbedDir, lang + ".mp4")
        stitch_audio(sentences, os.path.join(audioDir, lang), videoFile, outFile)

  print("Done")

In [44]:
dub("/content/drive/My Drive/GCP/Sample.mp4", "/content/drive/My Drive/GCP", "en", ["hi"])

Extracting audio from video
/content/drive/My Drive/GCP/Sample.wav
Wrote /content/drive/My Drive/GCP/Sample.wav
Connecting to Drive
Connected to Drive
Transcribing audio
Diarizing: False
[{'transcript': "It's taken me years of tragedy of losing myself inside, only to realize, what I must have always known that, you can be anything. You dream dream dream until your dreams come true act on your passion. And when your shot comes take it look fear in the face and embrace it. The time is now, the moment is now believed.", 'words': [{'word': "It's", 'start_time': 0.1, 'end_time': 0.4, 'speaker_tag': 0}, {'word': 'taken', 'start_time': 0.4, 'end_time': 0.7, 'speaker_tag': 0}, {'word': 'me', 'start_time': 0.7, 'end_time': 1.0, 'speaker_tag': 0}, {'word': 'years', 'start_time': 1.0, 'end_time': 1.5, 'speaker_tag': 0}, {'word': 'of', 'start_time': 1.5, 'end_time': 1.7, 'speaker_tag': 0}, {'word': 'tragedy', 'start_time': 1.7, 'end_time': 2.5, 'speaker_tag': 0}, {'word': 'of', 'start_time': 2.8, 

100%|██████████| 1096/1096 [00:01<00:00, 556.11it/s]


[MoviePy] Done.
[MoviePy] Writing video /content/drive/My Drive/GCP/dubbedVideos/hi.mp4


100%|██████████| 1190/1190 [00:03<00:00, 330.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/GCP/dubbedVideos/hi.mp4 

Done
